In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
import requests

from datetime import timedelta, date

import itertools

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
dates=[] #pasirinktų datų sąrašas nuskaitomiems duomenims

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2017, 1, 1) 
end_dt = date(2017, 12, 31)


for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
dates=[d for d in dates if d not in ['2014-07-12','2014-09-11','2014-03-03','2014-03-04','2014-03-05','2015-05-24',
                                     '2015-05-25','2015-05-26','2016-01-21','2016-04-10','2017-01-04','2017-01-05',
                                     '2017-02-22','2017-02-23','2017-02-24','2017-02-25','2017-02-26','2017-02-27',
                                     '2017-02-28','2017-03-01','2017-03-02','2017-03-03','2017-03-04','2017-03-05',
                                     '2017-03-06','2017-03-07']]
#2014-09-11 - cannot reshape array of size 294 into shape (22,14), nėra eilutės 'GELBĖJIMO DARBUS:'
#2014-07-12 - cannot reshape array of size 294 into shape (22,14), nėra eilutės 'b) buityje, vandenyje'
#2014-03-05 - cannot reshape array of size 309 into shape (22,14)
#2014-03-04 - cannot reshape array of size 295 into shape (22,14), nėra eilutės 'd) kitur'
#2014-03-03 - cannot reshape array of size 294 into shape (22,14), nėra eilutės 'e) chemija ir radiacija'
#2015-05-24-26 - cannot reshape array of size 309 into shape (22,14)
#2016-01-21 - .doc failas įkeltas
#2016-04-10 - cannot reshape array of size 294 into shape (22,14), nėra eilutės 'd) kitur'
#2017-01-04 - nėra eilutės 'Kiti išvykimai', vietoj jos 2 kartus 'Ekstremaliąsias situacijas'
#2017-01-05 - nėra eilutės pavadinimu 'Kiti išvykimai', neteisingas pavdinimas antrosios eilutės 'Ekstremaliąsias situacijas'
#2017-02-22-03-07 - cannot reshape array of size 330 into shape (22,14), 'nematomas stulpelis'

In [3]:
dfs1 = [] #Lentelių sąrašas paros duomenų lentelei apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus sukurti

for dateurl in dates:
    url = 'http://www.vpgt.lt/vpgt/m/m_events_summary/wfiles/e_'+dateurl+'.htm'
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    
    data0=[] #duomenų sąrašas
    
    for tr in rows:
        cols = tr.find_all('td')
        for td in cols:
            text = td.find_all(text=True)
            for t in text:
                data0.append(t)

                
    #Paros duomenys apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
    data0 = [w.replace('\r\n ', '') for w in data0]
    data1 = data0[data0.index('Ekstremaliąsias situacijas'):data0.index('2. Gaisrų aprašymas')]
    data1 = [a for a in data1 if a != '\n' and not('\xa0 ' in a)]    
    if data1[140] == 'd) pagalba ':
        data1[140:143] = [''.join(data1[140:143])]
    elif data1[150] == 'd) pagalba ':
        data1[150:153] = [''.join(data1[150:153])]
    elif data1[126] == 'd) pagalba ':
        data1[126:129] = [''.join(data1[126:129])]
            
    data1 = [w.replace('a) Gaisruose žuvusius ', 'a) Gaisruose žuvusius') for w in data1] #2017-02-22-03-07 colm=[1,2,3,14]
    #data1 = data1[0:308] #2015-05-24-26 2014-03-05 #2014-03-04 data1[0:294]
    
    df01 = pd.DataFrame(np.array(data1).reshape(22,14),
                        columns =["Duomenys apie:", "2017 nuo metų pr.", "2018 nuo metų pr.", "Iš viso per parą",
                                  "Vilniaus apskritis", "Kauno apskritis", "Klaipėdos apskritis", "Šiaulių apskritis",
                                  "Panevėžio apskritis", "Alytaus apskritis", "Utenos apskritis", "Marijampolės apskritis",
                                  "Tauragės apskritis", "Telšių apskritis"]) #2017-02-22-03-07 reshape(22,15), columns =[...'']
    
    colm=[1,2,3]  #2017-02-22-03-07 colm=[1,2,3,14]
    df01 = df01.drop(df01.columns[colm], axis=1)
    #df01 = df01.drop_duplicates() #2017-01-04
    #df01.loc[21,'Duomenys apie:'] = 'KITI IŠVYKIMAI'  #2017-01-05 
    df01 = df01.T.drop([2,6,13], axis=1) #2014-09-11 df01 = df01.T.drop([2,12], axis=1) #2014-03-03 drop([2,6,12]
    df01 = df01.rename(columns=df01.iloc[0]).drop(['Duomenys apie:']).reset_index()
    df01['Data'] = dateurl
    
    
    #df01['b) buityje, vandenyje'] = '' #2014-07-12
    #df01['e) chemija ir radiacija'] = '' #2014-03-03
    #df01['d) kitur'] = '' #2016-04-10 #2014-03-04
    #df01['KITI IŠVYKIMAI'] = [6, 5, 10, 5, 7, 3, 3, 1, 4, 2] #2017-01-04
    
    dfs1.append(df01)
    
    
# paros duomenų lentelė df1 apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
df1 = pd.concat(dfs1, ignore_index=True)
df1['Žuvę ne gaisre'], df1['Traumuoti ne gaisre'] = df1['Žuvę/traumuoti ne gaisre'].str.split('/',1).str
df1 = df1.drop(['Žuvę/traumuoti ne gaisre'], axis=1)
df1 = df1.rename(index=str, columns={"index": "Apskr", "Ekstremaliąsias situacijas": "EkstSitu", 
                                    "Ekstremaliuosius įvykius": "EkstIvyk", "GAISRUS:": "GAISRAI", 
                                    "a) Gaisruose žuvusius": "Zuve_gaisr", "b) traumuotus asmenis": "Trau_gaisr", 
                                    "a) transporto avarijose": "GD_tr_av", "b) vandenyje": "GD_vand", 
                                    "c) buityje": "GD_buit", "d) pagalba spec. tarnyboms": "GD_sp_trn", 
                                    "e) chemija ir radiacija": "GD_ch_rad", "f) kitus": "GD_kiti", 
                                    "a) gaisruose": "Isg_gaisr", "b) buityje, vandenyje": "Isg_buit_vand",
                                    "c) transporto avarijose": "Isg_tr_av", "d) kitur": "Isg_kitur",
                                    "Evakuoti žmonės": "Evak_zm", "Melagingi pranešimai": "MP", "KITI IŠVYKIMAI": "KI",
                                     "Žuvę ne gaisre": "Zuve_negsr", "Traumuoti ne gaisre": "Trau_negsr" })
df1 = df1[['Data','Apskr', 'EkstSitu', 'EkstIvyk', 'GAISRAI',
               'GD_tr_av', 'GD_vand', 'GD_buit', 'GD_sp_trn', 'GD_ch_rad', 'GD_kiti',
               'MP', 'KI', 'Evak_zm', 'Isg_gaisr', 'Isg_buit_vand', 'Isg_tr_av', 'Isg_kitur',
               'Trau_gaisr', 'Trau_negsr', 'Zuve_gaisr', 'Zuve_negsr']]


In [4]:
df1.to_csv('2017.csv', index=False, header=True, encoding="cp1257")